In [0]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [27]:
#This may need to be adjusted depending on where the files are.

df2016 = pd.read_pickle("../Census2016ElecDistrwResults.pkl")
df2011 = pd.read_pickle("../Census2016ElecDistrw2011Results.pkl")

df2016=df2016.rename(columns={"2015 Results": "Results"})
df2011=df2011.rename(columns={"2011 Results": "Results"})

df_results = df2016.append(df2011, ignore_index=True, sort=True)

df_results = df_results.drop(columns=["index"])

df_results.tail()


,English,French,Average total income in 2015 among recipients ($),Non-official languages,18 to 64 years (%),Average age of the population,Average household size,GEO_ID,GEO_UID,NAME,PROVINCE,Population density per square kilometre,"Population, 2016",Results
641,67375.0,2015.0,41264.0,39745.0,19.6,41.5,2.0,59035,2013A000459035,Vancouver East,British Columbia,5374.7,115724.0,NDP
642,56190.0,1600.0,59834.0,41335.0,18.5,41.7,2.1,59036,2013A000459036,Vancouver Granville,British Columbia,4620.3,103456.0,Conservative
643,29110.0,1565.0,53809.0,4210.0,None,39.1,2.3,60001,2013A000460001,Yukon,Yukon,0.1,35874.0,Conservative
644,31765.0,1175.0,64586.0,7625.0,None,34.9,2.7,61001,2013A000461001,Northwest Territories,Northwest Territories,0.0,41786.0,NDP
645,11020.0,595.0,50689.0,23340.0,None,27.7,3.6,62001,2013A000462001,Nunavut,Nunavut,0.0,35944.0,Conservative


In [28]:
# Prep the dataframe to run the optimization
df_calc = df_results.drop(columns = ["GEO_ID", "GEO_UID", "NAME", "PROVINCE"])

#drop the Greeen party  - forecasting for one seat won't work

df_calc = df_calc[df_calc.Results != 'Green']

df_calc.reset_index(inplace = True)


#Checking for parties - may be needed if we want to do further reductions.

#results = df_calc.loc[:, "Results"]

#for result in results:
  #print(result)

#Need to loop through the df - English, French, and non-official languages needs to be in %

i = 0
counter = df_calc.shape[0]

while i < counter:
  c = df_calc.at[i, '      English']
  d = df_calc.at[i, '      French']
  e = df_calc.at[i, '    Non-official languages']
  total = c+d+e
  c_new = c / total
  d_new = d / total
  e_new = e / total
  df_calc.at[i, '      English'] = c_new
  df_calc.at[i, '      French'] = d_new
  df_calc.at[i, '    Non-official languages'] = e_new
  i += 1
  
df_calc.head()

,index,English,French,Average total income in 2015 among recipients ($),Non-official languages,18 to 64 years (%),Average age of the population,Average household size,Population density per square kilometre,"Population, 2016",Results
0,0,0.992109,0.002280,48083.0,0.005611,9.3,42.4,2.4,13.4,86494.0,Liberal
1,1,0.992590,0.002379,38924.0,0.005031,12.6,47.4,2.3,4.4,74116.0,Liberal
2,2,0.991748,0.001949,38258.0,0.006302,15,46.5,2.3,2.0,77680.0,Liberal
3,3,0.864492,0.015284,56347.0,0.120224,6.5,37.0,2.7,0.1,27197.0,Liberal
4,4,0.986168,0.006595,38151.0,0.007237,14.9,46.5,2.3,2.3,86553.0,Liberal


# New Section

In [0]:

#Creating the results
y = df_calc["Results"]


In [102]:
#Creating the frame for x

df_X = df_calc.drop(columns = ["index", "Results"])

print(df_X.columns)

X = df_X [['      English', '      French',
       '    Average total income in 2015 among recipients ($)',
       '    Non-official languages', '  18 to 64 years (%)',
       'Average age of the population', 'Average household size',
       'Population density per square kilometre', 'Population, 2016']]

print(X.shape)


Index(['      English', '      French',
       '    Average total income in 2015 among recipients ($)',
       '    Non-official languages', '  18 to 64 years (%)',
       'Average age of the population', 'Average household size',
       'Population density per square kilometre', 'Population, 2016'],
      dtype='object')
(644, 9)


In [103]:
#Preprocessing done before the Train Test Split 

import sklearn.preprocessing as skprep
scaler = skprep.MinMaxScaler()
print(scaler)
scaler.fit(df_X)
X = scaler.transform(df_X)



MinMaxScaler(copy=True, feature_range=(0, 1))


In [0]:
#Need to eliminate the "nan"
#This is caused by the poverty rate - seems it is not tracked in the territories
#set value to 0.5
i = 0
X = X.astype('float32')
count = len(X)
while i < count:
  j = 0
  count2 = len(X[i])
  while j < count2:
    if np.isnan(X[i,j]):
      X[i,j]= 0.5
    #print(X_train[i,j])
    j += 1
      #print("nan", i, j)
      #X[i,j] = 0
    
  i += 1

# Create a Train Test Split

Use `koi_disposition` for the y values

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [106]:
X_train

array([[8.97821307e-01, 1.01964185e-02, 1.95159882e-01, ...,
        2.85714298e-01, 7.30957181e-05, 6.21123195e-01],
       [8.67838264e-01, 8.36563483e-03, 2.12104008e-01, ...,
        3.57142866e-01, 1.40568678e-04, 3.52020502e-01],
       [7.23237097e-01, 8.96685850e-03, 2.29874983e-01, ...,
        3.21428567e-01, 2.77313907e-02, 4.32376564e-01],
       ...,
       [8.75280678e-01, 3.44322040e-03, 1.72143996e-01, ...,
        3.21428567e-01, 1.69244700e-03, 6.13529265e-01],
       [3.17863792e-01, 3.19593796e-03, 1.10414535e-01, ...,
        3.57142866e-01, 1.27973735e-01, 5.41223228e-01],
       [8.77749860e-01, 2.23407261e-02, 1.87533170e-01, ...,
        2.85714298e-01, 2.75514612e-04, 5.13720810e-01]], dtype=float32)

# Pre-processing

Scale the data using the MinMaxScaler

In [107]:
#Checking the shapes
print(X_train.shape, y_train.shape)





(483, 9) (483,)


# Train the Support Vector Machine

In [0]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)

In [109]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.17723206, 0.14527795, 0.09667656, 0.10294921, 0.10163299,
       0.07967771, 0.06925835, 0.12159548, 0.10569969])

In [110]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, df_X.columns), reverse=True)

[(0.1772320633054837, '      English'),
 (0.14527795341626948, '      French'),
 (0.12159547836317593, 'Population density per square kilometre'),
 (0.10569968854632551, 'Population, 2016'),
 (0.10294921084005741, '    Non-official languages'),
 (0.10163299496699828, '  18 to 64 years (%)'),
 (0.09667655515592317,
  '    Average total income in 2015 among recipients ($)'),
 (0.07967770760868294, 'Average age of the population'),
 (0.0692583477970836, 'Average household size')]

In [111]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 0.7743271221532091
Testing Data Score: 0.40993788819875776
